# Welcome To GateStat !

Current version uses a Strong metric to give you accurate insights into your current performance of the gate prev year paper attempt. It is designed to reflect the **true potential** of the candidate, and how well the candidate has justified his/her attempt, by calculating accuracy and hit ratio. True Potential is found according to the tagged questions that one has left out during the exam. The terms involved are:

* **Total Score (tot)** : It is the total score/marks yeild according to the gain/loss due to marking scheme
* **Total Potential (potential)**  : It is the sum total of all the marks that one could have possibly attempted, i.e the perfect score/true potential
* **Total Scoreable (scorable)**  : It is the total marks that one has attempted to score.
* **Attempt Quality (quality)**  : It is the hit ratio or number of correct answers per question attempted, breaks a candidate's false confidence in the questions attempted.
* **Attempt Yeild   (attempt_m)**  : Actual marks gained out of maximum scoreable/attempted marks. i.e how much was attempting questions you attempted worth? How good your answers yeilded..
* **Potential Yeild (potential_m)**       : Actual marks gained out of true potential i.e How much of your potential you actually displayed?
* **Unused Potential(yeild) (unused_m)**  : Marks that is locked in your potential, that you can improve yourself to gain 
* **Unused Potential(no)    (unused_n)**  : Number of questions locked in your potential, that you can improve yourself to add to your performance

For full details on the implementation of the metric see [this](#StatGenerator) 

Set the correct year and name of AnswerKey pdf. Also keep the "myAnswers" pdf ready. <br>
For sample see <a href="myAnswers.xlsx">this</a>

In [6]:
name='CS1_Key'
year='2017'

# Setting constants
file=f'../prev_yr_papers/{year}/{name}.pdf'
path=f'../prev_yr_papers/{year}/myAnswers.xlsx'

Get myAnswers from the excel file

In [7]:
import pandas as pd
myAnswers=pd.read_excel(path,usecols=['Ans'],na_filter=False);myAnswers
# Get the Answers in real time from the GUI of the GateStat app

,Ans
0,D
1,B
2,A
3,B
4,B
...,...
60,
61,
62,
63,


Import everything and convert to pickle after getting data from table (If not done already)

In [11]:
# Use this segment only if the anskey pdf hasn't yet been converted to pickle file

import tabula as tb
import functools as ft
ansKey = tb.read_pdf(file, pages = "all", multiple_tables = True);
print(ansKey)
# ansKey = ft.reduce( lambda a,b: pd.concat([a,b],axis=0), ansKey )
# ansKey = ansKey.reset_index(drop=True)
# print(ansKey)
# ansKey.to_pickle(f'db/{str(year)}/{name}.pkl')


'pages' argument isn't specified.Will extract only from page 1 by default.


[]


Get data from Pickle File

In [6]:
try:
    ansKey = pd.read_pickle(f'../db/{str(year)}/{name}.pkl')
    ansKey = ansKey.rename(columns={'Key/Range':'Key'}) #some of them might not be the same name as Key/Range
    print('File reading success!!')
except :
    print('pickle file not found!\nExiting...')
    exit(-1)

File reading success!!


Run below cell only once or there will be duplicate columns of Ans

In [7]:
ansKey = pd.concat([ansKey,myAnswers],axis=1);ansKey

,Q.No.,Type,Section,Key,Marks,Ans
0,1,MCQ,GA,B,1,B
1,2,MCQ,GA,A,1,A
2,3,MCQ,GA,D,1,D
3,4,MCQ,GA,C,1,C
4,5,MCQ,GA,B,1,B
...,...,...,...,...,...,...
60,51,NAT,CS,32 to 32,2,*
61,52,NAT,CS,2 to 2,2,85
62,53,NAT,CS,85 to 85,2,
63,54,NAT,CS,144 to 144,2,


Build the scoreCard for me

### MARKING RESULTS COMPARING ANSWERS
We use generate code function to evaluate the code that is then used to evaluate the statistics of the student

In [8]:
import re
def regexMagic(keyItem) :
    '''Identifies and splits the ranges in the NAT questions'''
    f = re.split('\\s+to\\s+',keyItem)
    return [float(i) for i in f]

In [9]:
def generateCode(Key,Ans) :
    '''
    CODE RUBRIC BREAKDOWN
    
    * Needs Improvement Area (Code -1)
        |->Wrong Answers
    * Your Potentially Solveable Area (Code 2)
        |->Tagged 
    * Questions Unattempted (Code 0)
        |->Blank
    * Correct (Code 1)
    '''
    if  Ans is '' :
        return 0
    elif Ans is '*':
        return 2
    if Key.isalpha() :
        if Ans.lower() == Key.lower() :
            return 1
        else :
            return -1
    else :
        l,h = regexMagic(Key)
        if l <= float(Ans) <= h :
            return 1
        else :
            return -2
    return Key

In [10]:
Code = ansKey.apply(lambda x : generateCode(x['Key'],x['Ans']), axis=1).rename('Code')
marksheet = pd.concat([ansKey,Code],axis=1)
marksheet.tail(30)

,Q.No.,Type,Section,Key,Marks,Ans,Code
35,26,MCQ,CS,D,2,*,2
36,27,MCQ,CS,D,2,*,2
37,28,MCQ,CS,A,2,,0
38,29,MCQ,CS,A,2,C,-1
39,30,MCQ,CS,A,2,B,-1
40,31,MCQ,CS,C,2,A,-1
41,32,MCQ,CS,B,2,B,1
42,33,MCQ,CS,C,2,C,1
43,34,MCQ,CS,B,2,B,1
44,35,MCQ,CS,B,2,,0


Some helper functions coded below.. To follow flow of code go to [this](#FINDING-THE-STATS)

In [13]:
def findParameters(marksheet):
    '''
    Finds the seven important parameters 
    total marks of potential questions, correct questions, incorrect questions(numerical and mcq)
    total count of potential questions, correct questions, incorrect questions 
    '''
    mpot=0; mcorr=0; mincorr=0;mnincorr=0;
    npot=0; ncorr=0; nincorr=0;
    
    def iterate(code,mark):
        nonlocal mpot,mcorr,mincorr,npot,ncorr,nincorr,mnincorr 
        # nonlocal for refering to the variables immediately outside (similar to global)
        if code is 1 : 
            mcorr+=mark
            ncorr+=1
        elif code is 2 : 
            mpot+=mark
            npot+=1
        elif code is -1 : 
            mincorr+=mark
            nincorr+=1
        elif code is -2 :
            mnincorr+=mark
            nincorr+=1
    marksheet.apply(lambda x : iterate(x['Code'],x['Marks']),axis=1)
    return [mpot,mcorr,mincorr,npot,ncorr,nincorr,mnincorr]
#test
findParameters(marksheet)

[15, 46, 8, 9, 31, 10, 8]

### StatGenerator

In [14]:
def StatGenerator(marksheet):   
    '''
    Finds the six parameters and calculates the terms in the metric
    '''
    mpot,mcorr,mincorr,npot,ncorr,nincorr,mnincorr = findParameters(marksheet)
    # divide by zero error happens for some values
    
    tot         = mcorr - mincorr/3                         # Total Score     
    potential   = mpot + mcorr + mincorr + mnincorr         # Total Potential 
    scorable    = mincorr + mcorr + mnincorr                # Total Scoreable 
    quality     = ncorr/(ncorr+nincorr)*100                 # Attempt Quality 
    attempt_m   = tot/scorable*100                          # Attempt Yeild   
    potential_m = tot/potential*100                         # Potential Yeild 
    unused_m    = (mincorr + mpot + mnincorr)/potential*100 # Unused Potential(yeild) 
    unused_n    = (nincorr+npot)/(npot+ncorr+nincorr)*100   # Unused Potential(no)    
    
    return [ tot,
    potential,
    scorable,
    quality,
    attempt_m,
    potential_m,
    unused_m,
    unused_n ]

In [15]:
def findStat(marksheet):
    '''
    Finds the stats of CS Apti sections and driver function to calculate stats
    '''
    # first convert the marksheet into aptitude and cs divisions
    mark = marksheet.Section.str.contains(pat='.*CS.*') 
    # detect CS and mark those tuples
    Cs = marksheet[mark]
    Apti = marksheet[~mark]
    
    csStat = StatGenerator(Cs)
    aptiStat = StatGenerator(Apti)
    totStat = StatGenerator(marksheet)
    return [csStat,aptiStat,totStat]

### FINDING THE STATS

In [16]:
results = pd.DataFrame(findStat(marksheet),
             
             index=['CS','Apti','Total'],
             
             columns=['Total Score',
                    'Total Potential',
                    'Total Scoreable',
                    'Attempt Quality [%]',
                    'Attempt Yeild [%]',
                    'Potential Yeild [%]',
                    'Unused Potential(yeild) [%]',
                    'Unused Potential(no) [%]']
            ).transpose()
results

,CS,Apti,Total
Total Score,28.333333,15.0,43.333333
Total Potential,62.000000,15.0,77.000000
Total Scoreable,47.000000,15.0,62.000000
Attempt Quality [%],67.741935,100.0,75.609756
Attempt Yeild [%],60.283688,100.0,69.892473
Potential Yeild [%],45.698925,100.0,56.277056
Unused Potential(yeild) [%],50.000000,0.0,40.259740
Unused Potential(no) [%],47.500000,0.0,38.000000


### REFERENCE CODE(s) :
```
# Stack the DataFrames on top of each other
vertical_stack = pd.concat([survey_sub, survey_sub_last10], axis=0)

# Place the DataFrames side by side
horizontal_stack = pd.concat([survey_sub, survey_sub_last10], axis=1)

# tabula.convert_into(file, "key.csv",pages='all')
# tables :=list of pandas dataframe(s)

```